In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("..//Data/compas-scores-two-years.csv")
#display(df.columns)

In [ ]:
df

In [ ]:
display(df[['priors_count','juv_fel_count','juv_misd_count', 'juv_other_count','decile_score','is_recid','c_charge_degree', 'score_text']])

### Oppgave 1b

Omgjøre score_text til to kategorier, low --> 0, medium og high --> 1

In [3]:
#jobber med mindre dataframe
sdf = df[['sex', 'race', 'two_year_recid', 'score_text']]
sdf

,sex,race,two_year_recid,score_text
0,Male,Other,0,Low
1,Male,African-American,1,Low
2,Male,African-American,1,Low
3,Male,African-American,0,High
4,Male,Other,0,Low
...,...,...,...,...
7209,Male,African-American,0,Medium
7210,Male,African-American,0,Low
7211,Male,Other,0,Low
7212,Female,African-American,0,Low


In [ ]:
#don't think this is needed when using crosstable
#might affect efficiency in positive manner by this way- check later
"""
def pred(row):
    if row['high_risk'] == 1:
        if row['two_year_recid'] == 1:
            return 'TP'
        else:
            return 'FP'
    else:
        if row['two_year_recid'] == 1:
            return 'FN'
        else:
            return 'TN'
"""

In [4]:
sdf['high_risk'] = sdf['score_text'].apply(lambda score: 0 if score == 'Low' else 1) 
sdf['all'] = sdf['score_text'].apply(lambda x: 'All')
sdf
#sdf['truth_tab'] = sdf.apply(lambda row: pred(row), axis = 1)

<ipython-input-4-3f5e8fb2c19b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['high_risk'] = sdf['score_text'].apply(lambda score: 0 if score == 'Low' else 1)
<ipython-input-4-3f5e8fb2c19b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['all'] = sdf['score_text'].apply(lambda x: 'All')


,sex,race,two_year_recid,score_text,high_risk,all
0,Male,Other,0,Low,0,All
1,Male,African-American,1,Low,0,All
2,Male,African-American,1,Low,0,All
3,Male,African-American,0,High,1,All
4,Male,Other,0,Low,0,All
...,...,...,...,...,...,...
7209,Male,African-American,0,Medium,1,All
7210,Male,African-American,0,Low,0,All
7211,Male,Other,0,Low,0,All
7212,Female,African-American,0,Low,0,All


In [5]:
def crosstab(df):
    return pd.crosstab(df.high_risk, df.two_year_recid, margins=True, normalize=True)

In [6]:
def false_positive(cross_table, label):
    return cross_table.loc[(label, 1), 0]

In [7]:
def true_positive(cross_table, label):
    return cross_table.loc[(label, 1), 1]

In [8]:
def true_negative(cross_table, label):
    return cross_table.loc[(label, 0), 0]

In [9]:
def false_negative(cross_table, label):
    return cross_table.loc[(label, 0), 1]

In [29]:
#correct?
#using bayes theorem - ish
def p_recid_g_high(cross_table, label):
    return cross_table.loc[(label, 1), 1] / cross_table.loc[(label, 1), 'All']

In [ ]:
by_sex = sdf.groupby('sex')
ct_sex = by_sex.apply(crosstab)
ct_sex

In [ ]:
functions = [false_positive, false_negative, true_positive, false_positive, p_recid_g_high]
d = {}
for function in functions:
    inner_dict = {}
    for group in by_sex.groups:
        inner_dict[str(group)] = function(ct_sex, str(group))
    d[function.__name__] = inner_dict
d

In [ ]:
#CHECK IF ALL NUMBERS ARE CORRECT!
sum(sdf.high_risk)
#this sum is correct
a = ct_sex.loc[('Female', 0), 1]
a

In [ ]:
ct_sex.index.values
ct_sex.columns

In [ ]:
by_race = sdf.groupby('race')
#drop any group? What counts as 'black' here?
by_race.apply(crosstab)

In [18]:
groupobj = [sdf.groupby('all'), sdf.groupby('sex'), sdf.groupby('race')]

In [23]:
cross_tables = [group.apply(crosstab) for group in groupobj]
#cross_tables

In [30]:
functions = [false_positive, false_negative, true_positive, true_negative, p_recid_g_high]
d = {}
for function in functions:
    inner_dict = {}
    for sort, table in zip(groupobj, cross_tables):
        for group in sort.groups:
            inner_dict[str(group)] = round(function(table, str(group)), 3)
    d[function.__name__] = inner_dict
d

{'false_positive': {'All': 0.178,
  'Female': 0.206,
  'Male': 0.171,
  'African-American': 0.218,
  'Asian': 0.062,
  'Caucasian': 0.142,
  'Hispanic': 0.137,
  'Native American': 0.167,
  'Other': 0.095},
 'false_negative': {'All': 0.169,
  'Female': 0.14,
  'Male': 0.175,
  'African-American': 0.144,
  'Asian': 0.094,
  'Caucasian': 0.188,
  'Hispanic': 0.203,
  'Native American': 0.056,
  'Other': 0.239},
 'true_positive': {'All': 0.282,
  'Female': 0.217,
  'Male': 0.298,
  'African-American': 0.37,
  'Asian': 0.188,
  'Caucasian': 0.206,
  'Hispanic': 0.162,
  'Native American': 0.5,
  'Other': 0.114},
 'true_negative': {'All': 0.372,
  'Female': 0.437,
  'Male': 0.356,
  'African-American': 0.268,
  'Asian': 0.656,
  'Caucasian': 0.464,
  'Hispanic': 0.499,
  'Native American': 0.278,
  'Other': 0.552},
 'p_recid_g_high': {'All': 0.614,
  'Female': 0.513,
  'Male': 0.635,
  'African-American': 0.63,
  'Asian': 0.75,
  'Caucasian': 0.591,
  'Hispanic': 0.542,
  'Native American':